# Whisky Recommender
<b>Tentative Workflow</b>
1. Extract Nose, Taste, Finish text
2. Combine text from Step into 1 column
3. Remove punctuations and stopwords from output of Step 2
4. Tokenize output of Step 3
5. Lemmatize output of Step 4
6. Create W2V vectors on output of Step 5
7. Create W2V vector for a test review
8. Print the title of the vector in Step 6 that is closest to the vector in Step 7

## Import Libraries

In [ ]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('max_colwidth', 2000)

import string
import re
import nltk
from gensim.models import Word2Vec
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

## Load Data

In [2]:
data = pd.read_csv("whisky_clean.csv")

# For Machine Learning, we would only need the Title and Combined Text columns
data = data[['Title', 'Nose', 'Taste', 'Finish']]

# This ensures that even if there are non-string values in the columns, they will be converted to strings before concatenation.
data['combined_text'] = data['Nose'].astype(str) + " " + data['Taste'].astype(str) + " " + data['Finish'].astype(str)

# Check if the text gets combined correctly for each review
data.head()

,Title,Nose,Taste,Finish,combined_text
0,"Review #844: Cardinat 1995 27yr, 50%","Toffee, peanut brittle, musty wood, wood spice, vanilla, grape, golden caramel. Sweet toffee and fruits and a little nutty","Creamy caramel, smooth toasted oak, vanilla, grape, musty wood, sweet floral notes, gentle spice, slightly stale wood notes. Light, creamy and sweet. A nice balance of creamy caramel and fruits but the mouthfeel is a little too light and the wood is just a bit off.","Sweet caramel and cherry candy, vanilla, some gentle wood spice and slightly bitter wood notes creep in on the tail end","Toffee, peanut brittle, musty wood, wood spice, vanilla, grape, golden caramel. Sweet toffee and fruits and a little nutty Creamy caramel, smooth toasted oak, vanilla, grape, musty wood, sweet floral notes, gentle spice, slightly stale wood notes. Light, creamy and sweet. A nice balance of creamy caramel and fruits but the mouthfeel is a little too light and the wood is just a bit off. Sweet caramel and cherry candy, vanilla, some gentle wood spice and slightly bitter wood notes creep in on the tail end"
1,"Review #843: Cardinat 2003 19yr, 51.5%","Hot cinnamon cherry, brash toasted wood, caramel, big hot spice, tannic wood, some grape under all the spice.","Brash toasted wood, hot cinnamon and cherry, spice heat, light candy grape and caramel, bitter wood, almost nutty, floral notes. Nose doesn't lie this drinks hot and sweet. Feels unbalanced, not enough oak or fruits to overcome the heat","Toasted wood, caramel, hot spice, cinnamon, hot cherry. Some richer caramel wood notes on the finish save this a bit","Hot cinnamon cherry, brash toasted wood, caramel, big hot spice, tannic wood, some grape under all the spice. Brash toasted wood, hot cinnamon and cherry, spice heat, light candy grape and caramel, bitter wood, almost nutty, floral notes. Nose doesn't lie this drinks hot and sweet. Feels unbalanced, not enough oak or fruits to overcome the heat Toasted wood, caramel, hot spice, cinnamon, hot cherry. Some richer caramel wood notes on the finish save this a bit"
2,"Review #842: Cardinat 2007 15yr, 54.5%","Caramel, sweet toasted oak, honey graham cracker, tannic wood, vanilla, grape, tingly menthol spice","Hot cherry, sweet candy grape, caramel, honeyed floral notes, spice heat, antique wood, sweetly toasted oak, vanilla. Spicey and sweet but not overly hot.","Hot cherry and grape, sweet caramel, toasted wood, sweet florals, mellow spice","Caramel, sweet toasted oak, honey graham cracker, tannic wood, vanilla, grape, tingly menthol spice Hot cherry, sweet candy grape, caramel, honeyed floral notes, spice heat, antique wood, sweetly toasted oak, vanilla. Spicey and sweet but not overly hot. Hot cherry and grape, sweet caramel, toasted wood, sweet florals, mellow spice"
3,"Review #841: Dailuaine 1997 24yr ""Signatory Cask Strength,"" hogsheads 50.5%","Big waxy malt and clean cereal grain, sharp green apple, golden caramel, toasted wood with a slight tannic edge, tropical fruits, light wood spice","Clean sweet barley, oily and malty mouthfeel, big green apple, mango and tropical fruits, sugary dessert pastry, thick and bready and sweet but also some richer caramel and toasted wood giving it just a touch of darker woody heft. Great thick mouthfeel","Caramel and thick waxy malt, sugary barley, sweet toasted wood, some vanilla. A thick waxy and malty finish with some gritty barrel notes","Big waxy malt and clean cereal grain, sharp green apple, golden caramel, toasted wood with a slight tannic edge, tropical fruits, light wood spice Clean sweet barley, oily and malty mouthfeel, big green apple, mango and tropical fruits, sugary dessert pastry, thick and bready and sweet but also some richer caramel and toasted wood giving it just a touch of darker woody heft. Great thick mouthfeel Caramel and thick waxy malt, sugary barley, sweet toasted wood, some vanilla. A thick waxy and malty finish with some gritty barrel notes"
4,"Review #840: ""Russell's Re

## Preprocess Data

### Remove Punctuations

In [3]:
# string.punctuation comprises of a list of all punctuations
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [4]:
def remove_punct(text):
    # store character only if it is not a punctuation
    text_nopunct = "".join([char for char in text if char not in string.punctuation])
    return text_nopunct

data['combined_text_clean'] = data['combined_text'].apply(lambda x: remove_punct(x))

data.head()

,Title,Nose,Taste,Finish,combined_text,combined_text_clean
0,"Review #844: Cardinat 1995 27yr, 50%","Toffee, peanut brittle, musty wood, wood spice, vanilla, grape, golden caramel. Sweet toffee and fruits and a little nutty","Creamy caramel, smooth toasted oak, vanilla, grape, musty wood, sweet floral notes, gentle spice, slightly stale wood notes. Light, creamy and sweet. A nice balance of creamy caramel and fruits but the mouthfeel is a little too light and the wood is just a bit off.","Sweet caramel and cherry candy, vanilla, some gentle wood spice and slightly bitter wood notes creep in on the tail end","Toffee, peanut brittle, musty wood, wood spice, vanilla, grape, golden caramel. Sweet toffee and fruits and a little nutty Creamy caramel, smooth toasted oak, vanilla, grape, musty wood, sweet floral notes, gentle spice, slightly stale wood notes. Light, creamy and sweet. A nice balance of creamy caramel and fruits but the mouthfeel is a little too light and the wood is just a bit off. Sweet caramel and cherry candy, vanilla, some gentle wood spice and slightly bitter wood notes creep in on the tail end",Toffee peanut brittle musty wood wood spice vanilla grape golden caramel Sweet toffee and fruits and a little nutty Creamy caramel smooth toasted oak vanilla grape musty wood sweet floral notes gentle spice slightly stale wood notes Light creamy and sweet A nice balance of creamy caramel and fruits but the mouthfeel is a little too light and the wood is just a bit off Sweet caramel and cherry candy vanilla some gentle wood spice and slightly bitter wood notes creep in on the tail end
1,"Review #843: Cardinat 2003 19yr, 51.5%","Hot cinnamon cherry, brash toasted wood, caramel, big hot spice, tannic wood, some grape under all the spice.","Brash toasted wood, hot cinnamon and cherry, spice heat, light candy grape and caramel, bitter wood, almost nutty, floral notes. Nose doesn't lie this drinks hot and sweet. Feels unbalanced, not enough oak or fruits to overcome the heat","Toasted wood, caramel, hot spice, cinnamon, hot cherry. Some richer caramel wood notes on the finish save this a bit","Hot cinnamon cherry, brash toasted wood, caramel, big hot spice, tannic wood, some grape under all the spice. Brash toasted wood, hot cinnamon and cherry, spice heat, light candy grape and caramel, bitter wood, almost nutty, floral notes. Nose doesn't lie this drinks hot and sweet. Feels unbalanced, not enough oak or fruits to overcome the heat Toasted wood, caramel, hot spice, cinnamon, hot cherry. Some richer caramel wood notes on the finish save this a bit",Hot cinnamon cherry brash toasted wood caramel big hot spice tannic wood some grape under all the spice Brash toasted wood hot cinnamon and cherry spice heat light candy grape and caramel bitter wood almost nutty floral notes Nose doesnt lie this drinks hot and sweet Feels unbalanced not enough oak or fruits to overcome the heat Toasted wood caramel hot spice cinnamon hot cherry Some richer caramel wood notes on the finish save this a bit
2,"Review #842: Cardinat 2007 15yr, 54.5%","Caramel, sweet toasted oak, honey graham cracker, tannic wood, vanilla, grape, tingly menthol spice","Hot cherry, sweet candy grape, caramel, honeyed floral notes, spice heat, antique wood, sweetly toasted oak, vanilla. Spicey and sweet but not overly hot.","Hot cherry and grape, sweet caramel, toasted wood, sweet florals, mellow spice","Caramel, sweet toasted oak, honey graham cracker, tannic wood, vanilla, grape, tingly menthol spice Hot cherry, sweet candy grape, caramel, honeyed floral notes, spice heat, antique wood, sweetly toasted oak, vanilla. Spicey and sweet but not overly hot. Hot cherry and grape, sweet caramel, toasted wood, sweet florals, mellow spice",Caramel sweet toasted oak honey graham cracker tannic wood vanilla grape tingly menthol spice Hot cherry sweet candy grape caramel honeyed floral notes spice heat antique wood sweetly toasted oak vanilla Spicey and sweet but not overly hot Hot 

### Tokenization

In [5]:
def tokenize(text):
    # Matches any character that is neither alphanumeric nor underscore
    # Add a + just in case there are 2 or more spaces between certain words
    tokens = re.split('\W+', text)
    return tokens

data['combined_text_tokenized'] = data['combined_text_clean'].apply(lambda x: tokenize(x.lower()))

data.head()

,Title,Nose,Taste,Finish,combined_text,combined_text_clean,combined_text_tokenized
0,"Review #844: Cardinat 1995 27yr, 50%","Toffee, peanut brittle, musty wood, wood spice, vanilla, grape, golden caramel. Sweet toffee and fruits and a little nutty","Creamy caramel, smooth toasted oak, vanilla, grape, musty wood, sweet floral notes, gentle spice, slightly stale wood notes. Light, creamy and sweet. A nice balance of creamy caramel and fruits but the mouthfeel is a little too light and the wood is just a bit off.","Sweet caramel and cherry candy, vanilla, some gentle wood spice and slightly bitter wood notes creep in on the tail end","Toffee, peanut brittle, musty wood, wood spice, vanilla, grape, golden caramel. Sweet toffee and fruits and a little nutty Creamy caramel, smooth toasted oak, vanilla, grape, musty wood, sweet floral notes, gentle spice, slightly stale wood notes. Light, creamy and sweet. A nice balance of creamy caramel and fruits but the mouthfeel is a little too light and the wood is just a bit off. Sweet caramel and cherry candy, vanilla, some gentle wood spice and slightly bitter wood notes creep in on the tail end",Toffee peanut brittle musty wood wood spice vanilla grape golden caramel Sweet toffee and fruits and a little nutty Creamy caramel smooth toasted oak vanilla grape musty wood sweet floral notes gentle spice slightly stale wood notes Light creamy and sweet A nice balance of creamy caramel and fruits but the mouthfeel is a little too light and the wood is just a bit off Sweet caramel and cherry candy vanilla some gentle wood spice and slightly bitter wood notes creep in on the tail end,"[toffee, peanut, brittle, musty, wood, wood, spice, vanilla, grape, golden, caramel, sweet, toffee, and, fruits, and, a, little, nutty, creamy, caramel, smooth, toasted, oak, vanilla, grape, musty, wood, sweet, floral, notes, gentle, spice, slightly, stale, wood, notes, light, creamy, and, sweet, a, nice, balance, of, creamy, caramel, and, fruits, but, the, mouthfeel, is, a, little, too, light, and, the, wood, is, just, a, bit, off, sweet, caramel, and, cherry, candy, vanilla, some, gentle, wood, spice, and, slightly, bitter, wood, notes, creep, in, on, the, tail, end]"
1,"Review #843: Cardinat 2003 19yr, 51.5%","Hot cinnamon cherry, brash toasted wood, caramel, big hot spice, tannic wood, some grape under all the spice.","Brash toasted wood, hot cinnamon and cherry, spice heat, light candy grape and caramel, bitter wood, almost nutty, floral notes. Nose doesn't lie this drinks hot and sweet. Feels unbalanced, not enough oak or fruits to overcome the heat","Toasted wood, caramel, hot spice, cinnamon, hot cherry. Some richer caramel wood notes on the finish save this a bit","Hot cinnamon cherry, brash toasted wood, caramel, big hot spice, tannic wood, some grape under all the spice. Brash toasted wood, hot cinnamon and cherry, spice heat, light candy grape and caramel, bitter wood, almost nutty, floral notes. Nose doesn't lie this drinks hot and sweet. Feels unbalanced, not enough oak or fruits to overcome the heat Toasted wood, caramel, hot spice, cinnamon, hot cherry. Some richer caramel wood notes on the finish save this a bit",Hot cinnamon cherry brash toasted wood caramel big hot spice tannic wood some grape under all the spice Brash toasted wood hot cinnamon and cherry spice heat light candy grape and caramel bitter wood almost nutty floral notes Nose doesnt lie this drinks hot and sweet Feels unbalanced not enough oak or fruits to overcome the heat Toasted wood caramel hot spice cinnamon hot cherry Some richer caramel wood notes on the finish save this a bit,"[hot, cinnamon, cherry, brash, toasted, wood, caramel, big, hot, spice, tannic, wood, some, grape, under, all, the, spice, brash, toasted, wood, hot, cinnamon, and, cherry, spice, heat, light, candy, grape, and, caramel, bitter, wood, almost, nutty, floral, notes, nose, doesnt, lie, this, drinks, hot, and, sweet, feels, unbalanced, not, enough, oak, or, frui

### Remove Stopwords

In [6]:
import nltk

# NLTK(Natural Language Toolkit) in python has a list of stopwords stored in 16 different languages.
stopword = nltk.corpus.stopwords.words('english')

# if you get an error with the above code, comment the code above and run this & follow below directions:
# nltk.download()

<b>If you ran into issues with the above:</b>

1. Run <code>nltk.download()</code>. A new screen will pop up outside your Jupyter notebook. (It may be hidden behind other windows.)
2. Once this box opens up, click <code>all</code>, then <code>download</code>. Once this is done, restart your Jupyter notebook and try running the cell above again.

In [7]:
def remove_stopwords(tokenized_list):
    # Store in text only if word is not found in stopword i.e. it is not a stopword
    text = [word for word in tokenized_list if word not in stopword]
    return text

data['combined_text_nostop'] = data['combined_text_tokenized'].apply(lambda x: remove_stopwords(x))

data.head()

,Title,Nose,Taste,Finish,combined_text,combined_text_clean,combined_text_tokenized,combined_text_nostop
0,"Review #844: Cardinat 1995 27yr, 50%","Toffee, peanut brittle, musty wood, wood spice, vanilla, grape, golden caramel. Sweet toffee and fruits and a little nutty","Creamy caramel, smooth toasted oak, vanilla, grape, musty wood, sweet floral notes, gentle spice, slightly stale wood notes. Light, creamy and sweet. A nice balance of creamy caramel and fruits but the mouthfeel is a little too light and the wood is just a bit off.","Sweet caramel and cherry candy, vanilla, some gentle wood spice and slightly bitter wood notes creep in on the tail end","Toffee, peanut brittle, musty wood, wood spice, vanilla, grape, golden caramel. Sweet toffee and fruits and a little nutty Creamy caramel, smooth toasted oak, vanilla, grape, musty wood, sweet floral notes, gentle spice, slightly stale wood notes. Light, creamy and sweet. A nice balance of creamy caramel and fruits but the mouthfeel is a little too light and the wood is just a bit off. Sweet caramel and cherry candy, vanilla, some gentle wood spice and slightly bitter wood notes creep in on the tail end",Toffee peanut brittle musty wood wood spice vanilla grape golden caramel Sweet toffee and fruits and a little nutty Creamy caramel smooth toasted oak vanilla grape musty wood sweet floral notes gentle spice slightly stale wood notes Light creamy and sweet A nice balance of creamy caramel and fruits but the mouthfeel is a little too light and the wood is just a bit off Sweet caramel and cherry candy vanilla some gentle wood spice and slightly bitter wood notes creep in on the tail end,"[toffee, peanut, brittle, musty, wood, wood, spice, vanilla, grape, golden, caramel, sweet, toffee, and, fruits, and, a, little, nutty, creamy, caramel, smooth, toasted, oak, vanilla, grape, musty, wood, sweet, floral, notes, gentle, spice, slightly, stale, wood, notes, light, creamy, and, sweet, a, nice, balance, of, creamy, caramel, and, fruits, but, the, mouthfeel, is, a, little, too, light, and, the, wood, is, just, a, bit, off, sweet, caramel, and, cherry, candy, vanilla, some, gentle, wood, spice, and, slightly, bitter, wood, notes, creep, in, on, the, tail, end]","[toffee, peanut, brittle, musty, wood, wood, spice, vanilla, grape, golden, caramel, sweet, toffee, fruits, little, nutty, creamy, caramel, smooth, toasted, oak, vanilla, grape, musty, wood, sweet, floral, notes, gentle, spice, slightly, stale, wood, notes, light, creamy, sweet, nice, balance, creamy, caramel, fruits, mouthfeel, little, light, wood, bit, sweet, caramel, cherry, candy, vanilla, gentle, wood, spice, slightly, bitter, wood, notes, creep, tail, end]"
1,"Review #843: Cardinat 2003 19yr, 51.5%","Hot cinnamon cherry, brash toasted wood, caramel, big hot spice, tannic wood, some grape under all the spice.","Brash toasted wood, hot cinnamon and cherry, spice heat, light candy grape and caramel, bitter wood, almost nutty, floral notes. Nose doesn't lie this drinks hot and sweet. Feels unbalanced, not enough oak or fruits to overcome the heat","Toasted wood, caramel, hot spice, cinnamon, hot cherry. Some richer caramel wood notes on the finish save this a bit","Hot cinnamon cherry, brash toasted wood, caramel, big hot spice, tannic wood, some grape under all the spice. Brash toasted wood, hot cinnamon and cherry, spice heat, light candy grape and caramel, bitter wood, almost nutty, floral notes. Nose doesn't lie this drinks hot and sweet. Feels unbalanced, not enough oak or fruits to overcome the heat Toasted wood, caramel, hot spice, cinnamon, hot cherry. Some richer caramel wood notes on the finish save this a bit",Hot cinnamon cherry brash toasted wood caramel big hot spice tannic wood some grape under all the spice Brash toasted wood hot cinnamon and cherry spice heat light candy grape and caramel bitter wood almost nutty floral notes Nose doesnt lie this drinks hot and sweet Feels unbalanced not enough oak or fruits to o

### Lemmatization

In [8]:
# if you get an error with this cell, uncomment the line of code and re-run this cell
#nltk.download('wordnet')

wn = nltk.WordNetLemmatizer()

def lemmatizing(tokenized_text):
    # return list of all lemmatized words for their corresponding words in tokenized_text
    text = [wn.lemmatize(word) for word in tokenized_text]
    return text

data['combined_text_lemmatized'] = data['combined_text_nostop'].apply(lambda x: lemmatizing(x))

data.head()

,Title,Nose,Taste,Finish,combined_text,combined_text_clean,combined_text_tokenized,combined_text_nostop,combined_text_lemmatized
0,"Review #844: Cardinat 1995 27yr, 50%","Toffee, peanut brittle, musty wood, wood spice, vanilla, grape, golden caramel. Sweet toffee and fruits and a little nutty","Creamy caramel, smooth toasted oak, vanilla, grape, musty wood, sweet floral notes, gentle spice, slightly stale wood notes. Light, creamy and sweet. A nice balance of creamy caramel and fruits but the mouthfeel is a little too light and the wood is just a bit off.","Sweet caramel and cherry candy, vanilla, some gentle wood spice and slightly bitter wood notes creep in on the tail end","Toffee, peanut brittle, musty wood, wood spice, vanilla, grape, golden caramel. Sweet toffee and fruits and a little nutty Creamy caramel, smooth toasted oak, vanilla, grape, musty wood, sweet floral notes, gentle spice, slightly stale wood notes. Light, creamy and sweet. A nice balance of creamy caramel and fruits but the mouthfeel is a little too light and the wood is just a bit off. Sweet caramel and cherry candy, vanilla, some gentle wood spice and slightly bitter wood notes creep in on the tail end",Toffee peanut brittle musty wood wood spice vanilla grape golden caramel Sweet toffee and fruits and a little nutty Creamy caramel smooth toasted oak vanilla grape musty wood sweet floral notes gentle spice slightly stale wood notes Light creamy and sweet A nice balance of creamy caramel and fruits but the mouthfeel is a little too light and the wood is just a bit off Sweet caramel and cherry candy vanilla some gentle wood spice and slightly bitter wood notes creep in on the tail end,"[toffee, peanut, brittle, musty, wood, wood, spice, vanilla, grape, golden, caramel, sweet, toffee, and, fruits, and, a, little, nutty, creamy, caramel, smooth, toasted, oak, vanilla, grape, musty, wood, sweet, floral, notes, gentle, spice, slightly, stale, wood, notes, light, creamy, and, sweet, a, nice, balance, of, creamy, caramel, and, fruits, but, the, mouthfeel, is, a, little, too, light, and, the, wood, is, just, a, bit, off, sweet, caramel, and, cherry, candy, vanilla, some, gentle, wood, spice, and, slightly, bitter, wood, notes, creep, in, on, the, tail, end]","[toffee, peanut, brittle, musty, wood, wood, spice, vanilla, grape, golden, caramel, sweet, toffee, fruits, little, nutty, creamy, caramel, smooth, toasted, oak, vanilla, grape, musty, wood, sweet, floral, notes, gentle, spice, slightly, stale, wood, notes, light, creamy, sweet, nice, balance, creamy, caramel, fruits, mouthfeel, little, light, wood, bit, sweet, caramel, cherry, candy, vanilla, gentle, wood, spice, slightly, bitter, wood, notes, creep, tail, end]","[toffee, peanut, brittle, musty, wood, wood, spice, vanilla, grape, golden, caramel, sweet, toffee, fruit, little, nutty, creamy, caramel, smooth, toasted, oak, vanilla, grape, musty, wood, sweet, floral, note, gentle, spice, slightly, stale, wood, note, light, creamy, sweet, nice, balance, creamy, caramel, fruit, mouthfeel, little, light, wood, bit, sweet, caramel, cherry, candy, vanilla, gentle, wood, spice, slightly, bitter, wood, note, creep, tail, end]"
1,"Review #843: Cardinat 2003 19yr, 51.5%","Hot cinnamon cherry, brash toasted wood, caramel, big hot spice, tannic wood, some grape under all the spice.","Brash toasted wood, hot cinnamon and cherry, spice heat, light candy grape and caramel, bitter wood, almost nutty, floral notes. Nose doesn't lie this drinks hot and sweet. Feels unbalanced, not enough oak or fruits to overcome the heat","Toasted wood, caramel, hot spice, cinnamon, hot cherry. Some richer caramel wood notes on the finish save this a bit","Hot cinnamon cherry, brash toasted wood, caramel, big hot spice, tannic wood, some grape under all the spice. Brash toasted wood, hot cinnamon and cherry, spice heat, light candy grape and caramel, bitter wood, almost nutty, floral notes. Nose doesn't lie this drinks hot and sweet. Feels u

## Train Model

In [9]:
# Train the word2vec model
# vector_size = size of word vector. size input all dimensions that meet window param value
# window = number of words before and after the focus word that will be considered as context
# min_count = number of times word must appear in corpus in order to create a word vector
### Important - Model will only be trained on words that meet min_count so it may not learn all words as a result
w2v_model = Word2Vec(data["combined_text_lemmatized"], vector_size=100, window=5, min_count=2)

In [10]:
# create a list to store the average of all word vectors for each row of data
review_vect_list = []

# Iterate through each row in X train data
for index, row in data.iterrows():

    # Create a Word2Vec vector for each row (review) in train data 
    # By applying the W2V model on each word of each review, we get the word vector (embedding) for each word
    # Each row (review) will be represented by average of all vectors of all words in each row that model has trained on
    model_vector = (np.mean([w2v_model.wv.get_vector(token) for token in row['combined_text_lemmatized'] if token in w2v_model.wv], axis=0)).tolist()

    # Check if the line exists else it is vector of zeros
    if type(model_vector) is list:  
        review_vect_list.append(model_vector)
    else:
        review_vect_list.append([str(0) for i in range(100)])
        
word2vec_df = pd.DataFrame(review_vect_list)
word2vec_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
0,-0.074148,0.157858,0.087167,-0.121193,0.061908,-0.516805,0.179544,0.743524,-0.560690,-0.371377,0.044417,-0.201388,-0.236085,0.182328,0.138254,-0.329557,0.141401,-0.427552,-0.152097,-0.694602,-0.015417,0.167031,0.362637,0.015979,0.175418,0.039721,-0.286169,-0.240249,-0.367391,0.252999,0.422120,-0.034322,0.073917,-0.441445,-0.011302,0.399267,-0.052552,-0.241578,-0.148453,-0.518559,0.099149,-0.370323,-0.075306,0.192067,0.362776,-0.009956,-0.063153,-0.121752,0.231607,0.234188,-0.021180,-0.249177,0.015530,-0.013802,-0.106491,0.191362,0.190952,-0.088061,-0.258661,0.041343,-0.009105,0.081059,-0.051239,-0.077067,-0.498419,0.319942,0.335562,0.326416,-0.348644,0.374796,-0.099969,0.299451,0.309196,0.196712,0.328060,0.198825,-0.013697,0.156510,-0.220225,0.186443,-0.122083,-0.341248,-0.583224,0.545880,-0.367871,-0.009335,0.291948,0.137126,0.581818,0.288307,0.350067,0.292943,0.127384,0.042684,0.509049,0.394954,0.129078,-0.365101,0.177597,0.034769
1,-0.064676,0.157587,0.094005,-0.118643,0.062416,-0.525588,0.185034,0.769289,-0.579935,-0.370898,0.040168,-0.216419,-0.228684,0.202420,0.136236,-0.345054,0.142119,-0.429306,-0.162655,-0.736853,0.001985,0.171414,0.357334,0.015806,0.168759,0.019238,-0.275626,-0.257454,-0.387830,0.258837,0.442673,-0.027902,0.072646,-0.451846,-0.022529,0.419664,-0.052507,-0.255761,-0.144837,-0.544806,0.106460,-0.398258,-0.081572,0.200524,0.371779,-0.006146,-0.073955,-0.128445,0.225680,0.239130,-0.008645,-0.275937,-0.000500,-0.010422,-0.112302,0.204723,0.188988,-0.088761,-0.277739,0.038028,-0.000119,0.095973,-0.057043,-0.087386,-0.514764,0.344044,0.342961,0.326713,-0.367689,0.385578,-0.101246,0.307709,0.319028,0.187272,0.353565,0.202787,-0.015480,0.154629,-0.234163,0.192886,-0.117810,-0.343849,-0.619794,0.556625,-0.373673,-0.004069,0.285846,0.136774,0.607216,0.300629,0.364739,0.307713,0.124057,0.048048,0.524675,0.401452,0.145104,-0.380714,0.186978,0.028224
2,-0.062245,0.152682,0.082234,-0.116055,0.064923,-0.502458,0.175779,0.735112,-0.563956,-0.365121,0.050337,-0.198146,-0.223749,0.194131,0.143629,-0.329556,0.135233,-0.413947,-0.154838,-0.694170,-0.007556,0.153080,0.352065,0.013968,0.169388,0.020754,-0.264327,-0.233702,-0.370288,0.242221,0.427199,-0.037224,0.060918,-0.440781,-0.013529,0.396724,-0.048902,-0.234930,-0.141781,-0.512170,0.105460,-0.366976,-0.080254,0.195614,0.359493,-0.002541,-0.069872,-0.116452,0.220586,0.231748,-0.014760,-0.253887,0.003897,-0.012878,-0.105832,0.183022,0.173600,-0.089659,-0.266865,0.034210,-0.004559,0.087493,-0.043099,-0.085042,-0.489340,0.314213,0.328001,0.319329,-0.349648,0.371086,-0.097006,0.303870,0.300211,0.188065,0.333393,0.192169,-0.016931,0.149421,-0.217999,0.179229,-0.118637,-0.346832,-0.584060,0.530561,-0.364683,-0.017288,0.281149,0.126254,0.577050,0.293388,0.337650,0.287576,0.119946,0.046133,0.505722,0.383419,0.127186,-0.360261,0.175390,0.029797
3,-0.086468,0.150597,0.091973,-0.123844,0.054572,-0.503114,0.169573,0.717453,-0.522868,-0.359724,0.039788,-0.196886,-0.247059,0.159461,0.127641,-0.315124,0.142285,-0.420281,-0.146771,-0.651538,-0.024143,0.181107,0.364133,0.027094,0.171273,0.064435,-0.299257,-0.240343,-0.335233,0.255532,0.381257,-0.036682,0.092903,-0.418483,-0.011554,0.385637,-0.050695,-0.235467,-0.151874,-0.500114,0.087825,-0.350869,-0.066623,0.180289,0.340203,-0.020400,-0.056627,-0.131314,0.233714,0.233730,-0.027226,-0.228755,0.028583,-0.013505,-0.091971,0.197363,0.216274,-0.079111,-0.235016,0.047660,-0.017697,0.074601,-0.065864,-0.068319,-0.489784,0.310271,0.323806,0.308623,-0.325465,0.363812,-0.097699,0.273508,0.301074,0.196042,0.302943,0.188175,-0.005136,0.161895,-0.220670,0.190103,-0.123761,-0.311797,-0.552515,0.532663,-0.365765,0.007306,0.280225,0.147089,0.553715,0.263434,0.354

In [11]:
# Add 'sentence_vector' column to data DataFrame
data['sentence_vector'] = word2vec_df.values.tolist()
data

,Title,Nose,Taste,Finish,combined_text,combined_text_clean,combined_text_tokenized,combined_text_nostop,combined_text_lemmatized,sentence_vector
0,"Review #844: Cardinat 1995 27yr, 50%","Toffee, peanut brittle, musty wood, wood spice, vanilla, grape, golden caramel. Sweet toffee and fruits and a little nutty","Creamy caramel, smooth toasted oak, vanilla, grape, musty wood, sweet floral notes, gentle spice, slightly stale wood notes. Light, creamy and sweet. A nice balance of creamy caramel and fruits but the mouthfeel is a little too light and the wood is just a bit off.","Sweet caramel and cherry candy, vanilla, some gentle wood spice and slightly bitter wood notes creep in on the tail end","Toffee, peanut brittle, musty wood, wood spice, vanilla, grape, golden caramel. Sweet toffee and fruits and a little nutty Creamy caramel, smooth toasted oak, vanilla, grape, musty wood, sweet floral notes, gentle spice, slightly stale wood notes. Light, creamy and sweet. A nice balance of creamy caramel and fruits but the mouthfeel is a little too light and the wood is just a bit off. Sweet caramel and cherry candy, vanilla, some gentle wood spice and slightly bitter wood notes creep in on the tail end",Toffee peanut brittle musty wood wood spice vanilla grape golden caramel Sweet toffee and fruits and a little nutty Creamy caramel smooth toasted oak vanilla grape musty wood sweet floral notes gentle spice slightly stale wood notes Light creamy and sweet A nice balance of creamy caramel and fruits but the mouthfeel is a little too light and the wood is just a bit off Sweet caramel and cherry candy vanilla some gentle wood spice and slightly bitter wood notes creep in on the tail end,"[toffee, peanut, brittle, musty, wood, wood, spice, vanilla, grape, golden, caramel, sweet, toffee, and, fruits, and, a, little, nutty, creamy, caramel, smooth, toasted, oak, vanilla, grape, musty, wood, sweet, floral, notes, gentle, spice, slightly, stale, wood, notes, light, creamy, and, sweet, a, nice, balance, of, creamy, caramel, and, fruits, but, the, mouthfeel, is, a, little, too, light, and, the, wood, is, just, a, bit, off, sweet, caramel, and, cherry, candy, vanilla, some, gentle, wood, spice, and, slightly, bitter, wood, notes, creep, in, on, the, tail, end]","[toffee, peanut, brittle, musty, wood, wood, spice, vanilla, grape, golden, caramel, sweet, toffee, fruits, little, nutty, creamy, caramel, smooth, toasted, oak, vanilla, grape, musty, wood, sweet, floral, notes, gentle, spice, slightly, stale, wood, notes, light, creamy, sweet, nice, balance, creamy, caramel, fruits, mouthfeel, little, light, wood, bit, sweet, caramel, cherry, candy, vanilla, gentle, wood, spice, slightly, bitter, wood, notes, creep, tail, end]","[toffee, peanut, brittle, musty, wood, wood, spice, vanilla, grape, golden, caramel, sweet, toffee, fruit, little, nutty, creamy, caramel, smooth, toasted, oak, vanilla, grape, musty, wood, sweet, floral, note, gentle, spice, slightly, stale, wood, note, light, creamy, sweet, nice, balance, creamy, caramel, fruit, mouthfeel, little, light, wood, bit, sweet, caramel, cherry, candy, vanilla, gentle, wood, spice, slightly, bitter, wood, note, creep, tail, end]","[-0.07414751499891281, 0.15785843133926392, 0.08716709911823273, -0.12119288742542267, 0.06190824136137962, -0.5168047547340393, 0.1795443743467331, 0.7435243725776672, -0.5606901049613953, -0.371376633644104, 0.044416576623916626, -0.2013876885175705, -0.23608459532260895, 0.18232788145542145, 0.1382535994052887, -0.3295574486255646, 0.14140109717845917, -0.4275517761707306, -0.15209732949733734, -0.6946024298667908, -0.015416611917316914, 0.16703066229820251, 0.3626369833946228, 0.015979334712028503, 0.17541778087615967, 0.03972123563289642, -0.2861688435077667, -0.24024921655654907, -0.36739087104797363, 0.2529990077018738, 0.4221198260784149, -0.034321729093790054, 0.07391674071550369, -0.4414454400539398, -0.011301993392407894, 0.39926713705062866, -0.052552010864019394, 

## Make Predictions

In [17]:
# Function to get word vectors
def get_word_vectors(sentence):
    word_vectors = []
    for word in sentence.split():
        if word in w2v_model.wv:
            word_vectors.append(w2v_model.wv[word])
    return word_vectors

# Function to calculate sentence vector by averaging word vectors
def get_sentence_vector(sentence):
    word_vectors = get_word_vectors(sentence)
    if not word_vectors:
        return np.zeros(w2v_model.vector_size)
    return np.mean(word_vectors, axis=0)

# Function to find the closest sentence to the input text
def find_closest_sentence(input_text, data):
    input_vector = get_sentence_vector(input_text)
    max_similarity = -1
    closest_sentence = None
    
    for index, row in data.iterrows():
        sentence_vector = row['sentence_vector']
        similarity = cosine_similarity([input_vector], [sentence_vector])[0][0]
        if similarity > max_similarity:
            max_similarity = similarity
            closest_sentence = row['Title']
    
    return closest_sentence

In [22]:
# Example usage:
print("Enter your preference")
input_text = input()

closest_sentence = find_closest_sentence(input_text, data)
print("Closest Review:", closest_sentence)

Enter your preference
chicken rice is savoury
Closest Review: Review #844: Cardinat 1995 27yr, 50%
